In [1]:
# !pip install web3
# !pip install python-dotenv
# !pip install requests

## Imports

In [2]:
from web3 import Web3

# To read environment property file
import os
from dotenv import load_dotenv
from pathlib import Path

# To load ABI
import json

import requests
# For processing large numbers
from decimal import Decimal

## Constants

In [3]:
# Uniswap v3 Factory address
UNISWAP_V3_FACTORY = '0x1F98431c8aD98523631AE4a59f267346ea31F984'

# Etherscan endpoint
ETHERSCAN_ENDPOINT = 'https://api.etherscan.io/api'

# Token0 (USDC) token contract
TOKEN0 = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'

# Token1 (WETH) token contract
TOKEN1 = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'

# Fees 0.05% == 500
FEES = 500

# Uniswap v3 Pool  ABI path
POOL_ABI = 'abi/uniswap_pool_abi.json'

# Path to the ERC20 ABI
ERC20_ABI = 'abi/erc20.json'

## Load environment variables

In [4]:
dotenv_path = Path('.env/price')
load_dotenv(dotenv_path=dotenv_path)
PROVIDER_URL = os.getenv('PROVIDER_URL')
ETHERSCAN_API_KEY = os.getenv('ETHERSCAN_API_KEY')

In [5]:
def get_univ3_factory_abi() -> str:
    """ Returns Uniswap v3 factory ABI
        
    Returns:
    str
        The factory ABI or None for errors
    """
    # Get ABI of contract; use etherscan API for verifiable smart contracts ABI
    params = {'module':'contract', 'action':'getabi', 'address':{UNISWAP_V3_FACTORY}, 'apikey':{ETHERSCAN_API_KEY}}
    response = json.loads(requests.get(url=ETHERSCAN_ENDPOINT, params=params).text)
    if response['status'] == '1':
        return json.loads(response['result'])
    return None

## Get Token Prices
### Reference: [A Primer on Uniswap v3 Math: As Easy As 1, 2, v3](https://blog.uniswap.org/uniswap-v3-math-primer)

In [6]:
def get_price(sqrt_price_x96:int, decimal0:int, decimal1:int):
    """ Returns Get the two token prices of the pool
    Parameters:
    sqrt_price_x96 : int
        from the pool's slot0 call
    decimal0: int
        number of decimals for token0
    decimal1: int
        number of decimals for token1

    Returns:
    tuple
        A tuple consisting of token0 and token1 prices
    """
    buy_one_of_token0 = round(((sqrt_price_x96 / Decimal(2)**96)**2) /
                              (Decimal(10)**decimal1 / Decimal(10)**decimal0), decimal1)    
    buy_one_of_token1 = round((1 / buy_one_of_token0), decimal0)
    
    return (buy_one_of_token0, buy_one_of_token1)

In [7]:
factory_abi = get_univ3_factory_abi()
assert factory_abi != None, 'Factory ABI not available'

# Load the previously downloaded pool ABI
with open(POOL_ABI) as f:
    pool_abi = json.load(f)
assert pool_abi != None, 'Pool ABI not available'

# Load ERC20 ABIs to get decimails for token0 and token1
with open(ERC20_ABI) as f:
    erc20_abi = json.load(f)
assert erc20_abi != None, 'ERC20 ABI not available'

# Create factory contract
web3 = Web3(Web3.HTTPProvider(PROVIDER_URL))
factory_contract = web3.eth.contract(address=web3.to_checksum_address(value=UNISWAP_V3_FACTORY), abi=factory_abi)
assert factory_contract != None, 'Factory contract does not exist'

# Get pool contract for token0 and token1
pool_address = factory_contract.functions.getPool(TOKEN1, TOKEN0, FEES).call()
pool_contract = web3.eth.contract(address=pool_address, abi=pool_abi)
assert pool_contract != None, 'Pool contract does not exist for token0 and token1'

## Call Get Price

In [8]:
def display_current_price() -> None:
    """ Displays the current token0 and token1 prices for the pool. This
    pool has two tokens: TOKEN0 and TOKEN1 with FEES
    """    
    token0_address = pool_contract.functions.token0().call()
    token0_contract = web3.eth.contract(address=token0_address, abi=erc20_abi)
    decimal0 = token0_contract.functions.decimals().call()
    
    token1_address = pool_contract.functions.token1().call()
    token1_contract = web3.eth.contract(address=token1_address, abi=erc20_abi)
    decimal1 = token1_contract.functions.decimals().call()
    
    sqrtx96 = pool_contract.functions.slot0().call()[0]
    
    token0_sym = token0_contract.functions.symbol.call()
    token1_sym = token1_contract.functions.symbol.call()
    
    # We have all the data ready to call get_price
    token0_price, token1_price = get_price(sqrt_price_x96=sqrtx96,
                                           decimal0=decimal0, decimal1=decimal1)
    print('price of {token0} in value of {token1} : {price}'.format(
        token0=token0_sym, token1=token1_sym, price=token0_price))
    print('price of {token1} in value of {token0} : {price}'.format(
        token1=token1_sym, token0=token0_sym, price=token1_price))

In [9]:
display_current_price()

price of USDC in value of WETH : 0.000500840936136297
price of WETH in value of USDC : 1996.641903


## Swap price - Approach 1

In [10]:
def display_swap_price_1(processed_log:dict, pool_abi:str, erc20_abi:str) -> None:
    """ Displays swap price - method 1
    Parameters:
    processed_log : dict
        A dictionary containing processed log items
    pool_abi:
        Pool ABI
    erc20_abi:
        ERC20 ABI
    """    
    # print(processed_log)
    pool_address = processed_log['address']
    pool_contract = web3.eth.contract(address=pool_address, abi=pool_abi)
    assert pool_contract != None, 'Pool contract must exist'

    sqrt_priceX96 = processed_log['args']['sqrtPriceX96']
    
    # Get decimals from token contracts
    token0_address = pool_contract.functions.token0().call()
    token0_contract = web3.eth.contract(address=token0_address, abi=erc20_abi)
    decimal0 = token0_contract.functions.decimals().call()
    token0_sym = token0_contract.functions.symbol.call()
    
    token1_address = pool_contract.functions.token1().call()
    token1_contract = web3.eth.contract(address=token1_address, abi=erc20_abi)
    decimal1 = token1_contract.functions.decimals().call()
    token1_sym = token1_contract.functions.symbol.call()
    
    # We have all the data ready to call get_price
    token0_price, token1_price = get_price(sqrt_price_x96=sqrt_priceX96, decimal0=decimal0, decimal1=decimal1)
    print('price of {token0} in value of {token1} : {price}'.format(
        token0=token0_sym, token1=token1_sym, price=token0_price))
    print('price of {token1} in value of {token0} : {price}'.format(
        token1=token1_sym, token0=token0_sym, price=token1_price))

In [11]:
tran_hash='0x8bab544f6f87449d25b9e4cef8e2d59f1771050b10cae2c17af1f35eceeffdf9'
receipt = web3.eth.get_transaction_receipt(tran_hash)
log = [x for x in receipt.logs if x['logIndex'] == 59][0]
processed_log = pool_contract.events['Swap']().process_log(log)
display_swap_price_1(processed_log=processed_log, pool_abi=pool_abi, erc20_abi=erc20_abi)

price of USDC in value of WETH : 0.000440269867806333
price of WETH in value of USDC : 2271.334182


## Swap price - Approach 2

In [12]:
def display_swap_price_2(processed_log:dict, pool_abi:str, erc20_abi:str) -> None:
    """ Displays swap price - method 1
    Parameters:
    processed_log : dict
        A dictionary containing processed log items
    pool_abi:
        Pool ABI
    erc20_abi:
        ERC20 ABI
    """
    # print(processed_log)
    pool_address = processed_log['address']
    pool_contract = web3.eth.contract(address=pool_address, abi=pool_abi)
    assert pool_contract != None, 'Pool contract must exist'
    
    # Get decimals from token contracts
    token0_address = pool_contract.functions.token0().call()
    token0_contract = web3.eth.contract(address=token0_address, abi=erc20_abi)
    decimal0 = token0_contract.functions.decimals().call()
    token0_sym = token0_contract.functions.symbol.call()
    
    token1_address = pool_contract.functions.token1().call()
    token1_contract = web3.eth.contract(address=token1_address, abi=erc20_abi)
    decimal1 = token1_contract.functions.decimals().call()
    token1_sym = token1_contract.functions.symbol.call()
    
    amount0 = processed_log['args']['amount0']/pow(10, decimal0)
    amount1 = processed_log['args']['amount1']/pow(10, decimal1)
       
    if amount0 < 0:
        print('Sell {amount1} {symbol1} for {amount0} {symbol0} at {price}'.format(
            amount1=amount1, symbol1=token1_sym, amount0=(amount0 * -1), symbol0=token0_sym,
            price=((amount0 * -1) / amount1)))
    else:
        print('Buy {amount1} {symbol1} for {amount0} {symbol0} at {price}'.format(
            amount1=(amount1 * -1), symbol1=token1_sym, amount0=amount0, symbol0=token0_sym,
            price=((amount1 * -1) / amount0)))

In [13]:
tran_hash='0x8bab544f6f87449d25b9e4cef8e2d59f1771050b10cae2c17af1f35eceeffdf9'
receipt = web3.eth.get_transaction_receipt(tran_hash)
log = [x for x in receipt.logs if x['logIndex'] == 59][0]
processed_log = pool_contract.events['Swap']().process_log(log)
display_swap_price_2(processed_log=processed_log, pool_abi=pool_abi, erc20_abi=erc20_abi)

Sell 1.4703945749634444 WETH for 3340.659443 USDC at 2271.947611805527


In [14]:
tran_hash='0x6704da4b05ed0e8d2ed2cd428777a11383994019427e46dc32af4bc81bdc87c2'
receipt = web3.eth.get_transaction_receipt(tran_hash)
log = [x for x in receipt.logs if x['logIndex'] == 169][0]
processed_log = pool_contract.events['Swap']().process_log(log)
display_swap_price_2(processed_log=processed_log, pool_abi=pool_abi, erc20_abi=erc20_abi)

Sell 3948.739085 USDT for 2.022856414386138 WETH at 0.0005122790771541031


In [15]:
tran_hash='0x6704da4b05ed0e8d2ed2cd428777a11383994019427e46dc32af4bc81bdc87c2'
receipt = web3.eth.get_transaction_receipt(tran_hash)
log = [x for x in receipt.logs if x['logIndex'] == 166][0]
processed_log = pool_contract.events['Swap']().process_log(log)
display_swap_price_2(processed_log=processed_log, pool_abi=pool_abi, erc20_abi=erc20_abi)

Buy 3948.739085 USDT for 3947.826815 USDC at 1.0002310815653144
